# Dependency distance calculation on a ConLL file


This notebook shows how to calculate the dependency distance of a document. It assumes that the document has already been parsed into the ConLL format. To parse your documents into ConLL format, use [Spacy](https://spacy.io), [UDPipe](https://ufal.mff.cuni.cz/udpipe), Google's [Syntaxnet](https://github.com/tensorflow/models/tree/master/research/syntaxnet) (harder to set up, but better quality), or any other parser.

The document in this example has two sentences. These have been parsed and tagged by UDPipe. Here's what the format looks like:

In [96]:
doc = """
# newdoc id = doc1
# newpar
# sent_id = 1
# text = I eat the pizza.
1	I	I	PRON	PRP	Case=Nom|Number=Sing|Person=1|PronType=Prs	2	nsubj	_	_
2	eat	eat	VERB	VBP	Mood=Ind|Tense=Pres|VerbForm=Fin	0	root	_	_
3	the	the	DET	DT	Definite=Def|PronType=Art	4	det	_	_
4	pizza	pizza	NOUN	NN	Number=Sing	2	obj	_	SpaceAfter=No
5	.	.	PUNCT	.	_	2	punct	_	_

# sent_id = 2
# text = The pizza, which I liked, was eaten by me.
1	The	the	DET	DT	Definite=Def|PronType=Art	2	det	_	_
2	pizza	pizza	NOUN	NN	Number=Sing	9	nsubj:pass	_	SpaceAfter=No
3	,	,	PUNCT	,	_	2	punct	_	_
4	which	which	PRON	WDT	PronType=Rel	6	obj	_	_
5	I	I	PRON	PRP	Case=Nom|Number=Sing|Person=1|PronType=Prs	6	nsubj	_	_
6	liked	like	VERB	VBD	Mood=Ind|Tense=Past|VerbForm=Fin	2	acl:relcl	_	SpaceAfter=No
7	,	,	PUNCT	,	_	9	punct	_	_
8	was	be	AUX	VBD	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	9	aux:pass	_	_
9	eaten	eat	VERB	VBN	Tense=Past|VerbForm=Part|Voice=Pass	0	root	_	_
10	by	by	ADP	IN	_	11	case	_	_
11	me	I	PRON	PRP	Case=Acc|Number=Sing|Person=1|PronType=Prs	9	obl	_	SpaceAfter=No
12	.	.	PUNCT	.	_	9	punct	_	SpacesAfter=\n

"""

Sentences are separated by empty lines, and each line represents one word/token. Each lines contains 10 fields, which are tab-separated. The fields are further described in the ([documentation](http://universaldependencies.org/format.html)). Lines starting with `#` contain metadata for the following sentence.

In a first step, we parse this document into a Pandas data frame. Because of the metadata, this is rather complex. A quick, but probably not very robust solution, is this:

In [97]:
import pandas as pd

def parse_conll(conll):
    lines = conll.split('\n')
    data = []  # hold the rows of the data frame
    doc_ids = []  # holds document ids
    sentence_ids = [1]  # holds sentence ids
    for line in lines:
        line = line.strip()
        # the conll file might contain further annotations, 
        # these lines start with #
        if line.startswith('# newdoc id ='):
            doc_id = line.split('# newdoc id = ', 1)[1].strip()
            doc_ids.append(doc_id)
            sentence_ids = [1]
        elif line=='# newdoc':
            doc_ids.append(len(doc_ids)+1)
            sentence_ids = [1]
        elif line.startswith('# sent_id ='):
            sent_id = line.split('# sent_id = ', 1)[1].strip()
            # replace sentence id
            sentence_ids[-1] = sent_id
        elif line.startswith('#'):
            continue # ignore other metadata
        # the line is empty, start a new sentence
        elif not line: 
            sentence_ids.append(len(sentence_ids)+1)
        # the line has data, split by tab and add
        else:
            doc_id = doc_ids[-1] if doc_ids else None
            data.append((doc_id, sentence_ids[-1], *line.split('\t')))
            
    # turn into data frame
    cols = ['docid', 'sentid', 'id', 'token', 'lemma', 'upos',
            'xpos', 'feats', 'head', 'deprel', 'deps', 'misc']
    df = pd.DataFrame(data, columns=cols)
    
    return(df)

doc_df = parse_conll(doc)

The data frame `doc_df` now contains 19 rows (one for each token), and 12 columns. For dependency distance calculation, we only need a few columns:

In [98]:
doc_df[['docid', 'sentid', 'id', 'token', 'upos', 'head']]

,docid,sentid,id,token,upos,head
0,doc1,1,1,I,PRON,2
1,doc1,1,2,eat,VERB,0
2,doc1,1,3,the,DET,4
3,doc1,1,4,pizza,NOUN,2
4,doc1,1,5,.,PUNCT,2
5,doc1,2,1,The,DET,2
6,doc1,2,2,pizza,NOUN,9
7,doc1,2,3,",",PUNCT,2
8,doc1,2,4,which,PRON,6
9,doc1,2,5,I,PRON,6


The column `head` refers to the syntactic link between the token and its head. For instance, in the first sentence, "now" is linked to the token with id number `5`, which is the token "post". Dependency distance for each token is defined as the absolute distance between the token's id and the id of it's head. Each sentence contains exactly one "root", which in the first sentence is "post" (with a head of 0). Its dependency distance is defined as 0. The total mean dependency distance (MDD) for a sentence is defined as 

$$
\text{MDD}(s) = \frac{1}{N_{s}-1}\sum\limits_{i=1}^n |\text{DD}_{i}|,
$$

where $N_{s}$ refers the number of tokens in the sentence, and $DD_{i}$ refers to the dependency distance of the i-th token. To calculate MDD for the whole document,

$$
\text{MDD}_{\text{document}} = \frac{1}{N-S}\sum\limits_{i=1}^n |\text{DD}_{i}|,
$$

where $N$ refers to the the total number of tokens in the document, and $S$ refers to the total number of sentences. This weights all tokens in the document equally. (Note that this is different from weighting the MDD of the individual sentences).

For punctuation, the dependency distance is not defined. This would suggest to remove all rows with punctuation, which is reasonable for the first sentence. However, in the second sentence, the removal of the comma (token 4), leads to a gap in the token id. The calculation of MDD has to take this gap into account and correct both the token id and the `head` column.

Here's a way to achieve this:

In [105]:
from itertools import chain

def calc_mdd(d, by_sentence=False):
    # remove punctuation
    d = d[d['upos'].str.lower() != 'punct'].copy()
    # generate a new id
    doc_sent_n = list(d.groupby(['docid', 'sentid'], sort=False).size())
    d['new_id'] = list(chain.from_iterable([range(1, l+1) for l in doc_sent_n]))
    # generate correspondence between old id and new id
    # to correct the head column
    ref = d[['docid', 'sentid', 'id', 'new_id']]
    ref.columns = ['docid', 'sentid', 'head', 'new_head']
    # merge the correspondence back to the data
    d = pd.merge(d, ref, how='left', on=['docid', 'sentid', 'head'])
    # calculate DD for each token
    d['dd'] = (d['new_head'] - d['new_id']).abs()
    if by_sentence==True:
        agg = d.groupby(['docid', 'sentid']).agg(
            {'dd': 'sum', 'token': 'count'}).reset_index()
        agg['mdd'] = agg['dd'] / (agg['token'] - 1)
        agg.columns = ['docid', 'sentid', 'sum_dd', 'n_tokens', 'mdd']
    else:
        agg = d.groupby(['docid']).agg(
            {'dd': 'sum', 'token': 'count', 'sentid': 'nunique'}).\
            reset_index()
        agg['mdd'] = agg['dd'] / (agg['token'] - agg['sentid'])
        agg.columns = ['docid', 'sum_dd', 'n_tokens', 'n_sents', 'mdd']
    return(agg)

In [106]:
calc_mdd(doc_df)

,docid,sum_dd,n_tokens,n_sents,mdd
0,doc1,20.0,13,2,1.818182


In [104]:
calc_mdd(doc_df, by_sentence=True)

,docid,sentid,sum_dd,n_tokens,mdd
0,doc1,1,4.0,4,1.333333
1,doc1,2,16.0,9,2.000000


In [109]:
# make sure that the results are the same if we take just one sentence
calc_mdd(doc_df[doc_df.sentid=="2"], by_sentence=True)

,docid,sentid,sum_dd,n_tokens,mdd
0,doc1,2,16.0,9,2.0
